# Задание 6

In [3]:
import numpy as np
%matplotlib inline
%matplotlib widget
import matplotlib.pyplot as plt
from typing import Callable
import pandas as pd
from random import uniform as rnd
import scipy.integrate as integrate

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Код из [Задания 5](https://github.com/astatochek/Computational_Workshop_4_sem/blob/main/5.1%20New%20Type%20of%20Integrals/note.ipynb)

In [23]:
# полиномы Лежандра (изначально P0 и P1)
legendre_polynomials = [lambda x: 1., lambda x: x]
# функция получения многочлена Лежандра степени n
def get_Legendre(n: int) -> Callable:
    def constructor(f1: Callable, f2: Callable, k: int) -> Callable:
        def func(x: float) -> float:
            return f1(x) * x * (2 * k - 1) / k - f2(x) * (k - 1) / k
        return func
    while len(legendre_polynomials) <= n:
        i = len(legendre_polynomials)
        next_func = constructor(legendre_polynomials[i-1], legendre_polynomials[i-2], i)
        legendre_polynomials.append(next_func)
    return legendre_polynomials[n]
# величина невязки в методе секущих
eps = 1.e-12
# метод секущих
def secant_method(f: Callable) -> np.ndarray:
    segments = []
    # функция для получения разбиения отрезка [a, b]
    def get_partition(n: float, a: float, b: float) -> None:
        h = (b - a) / n
        left = a
        for i in range(int(n)):
            right = a + (i + 1) * h
            if f(left) * f(right) < 0:
                segments.append([left, right])
            if f(right) != 0:
                left = right
    # получения разбиения для отрезка [-1, 1] с сохранением результатов в segments
    get_partition(1.e3, -1, 1)
    # функция для получения случайного числа из интервала (a, b)
    def get_random(a: float, b: float) -> float:
        x = rnd(a, b)
        while x == a or x == b:
            x = rnd(a, b)
        return x
    # сам метод секущих
    def pure_secants(s: list[float]) -> float:
        x0 = s[0]
        x1 = get_random(x0, s[1])
        x2 = x1 - (f(x1) / (f(x1) - f(x0))) * (x1 - x0)
        while abs(x2 - x1) >= eps:
            x2, x1, x0 = x2 - (f(x2) / (f(x2) - f(x1))) * (x2 - x1), x2, x1
        return x2
    # получение корней на сегментах методом секущих
    hubs = np.array([pure_secants(segment) for segment in segments])
    return hubs
# получение коэффициентов КФ Гаусса
def get_gauss_coefficients(hub_list: np.ndarray, n: int) -> np.ndarray:
    Pn1 = get_Legendre(n - 1)
    return np.array([2 * (1 - x**2) / (n * Pn1(x))**2 for x in hub_list])
# вывод коэффициентов и узлов КФ Гаусса степени n
def get_hubs_coefficients(n: int) -> pd.DataFrame:
    dfs = {}
    P = get_Legendre(n)
    hubs = secant_method(P)
    coefficients = get_gauss_coefficients(hubs, n)
    dfs[f"Узлы для N={n}"] = hubs
    dfs[f"Коэффициенты для N={n}"] = coefficients
    return pd.DataFrame(dfs, index=[f"X{i+1}" for i in range(n)])

 ### Реализация составной КФ Гаусса

In [33]:
def composite_gauss(f: Callable, a: float, b: float, n: int, m: int) -> float:
    legendre = get_Legendre(n)
    coefficients = get_gauss_coefficients(secant_method(legendre), n)
    hubs = secant_method(legendre)
    S = 0
    h = (b - a) / m
    for i in range(m):
        hub_list = np.array([(2*a + h * (t + 2*i+1)) / 2 for t in hubs])
        S += np.sum([coefficients[j] * f(hub_list[j]) for j in range(n)])
    return h * S / 2

### Вычисление $\displaystyle\int_a^b\rho(x)f(x)~dx$ при $a=0$, $b=1$, $\rho(x) = \sqrt{1-x}$, $f(x) = \sin(x)$

In [62]:
n = int(input('Введите число узлов для КФ Гаусса'))
m = int(input('Введите число промежутков деления'))
a = 0
b = 1
phi = lambda x: np.sqrt(1 - x) * np.sin(x)
data = {}
Expected = 0.2502016951430141493461499310819776539790258894746767857547317579
Value = composite_gauss(phi, a, b, n, m)
print(f'N = {n}, M = {m}')
print('Value:', Value)
print('Expected:', Expected)
print('Error:', abs(Value - Expected))
# print(f'(Expected Error: {Error})')
size = 5
for j in range(size):
    fragmentation = int(m * (j + 1 + size // 2) / size)
    Errors = []
    for i in range(size):
        degree = int(n * (i + 1 + size // 2) / size)
        val = composite_gauss(phi, a, b, degree, fragmentation)
        Errors.append(f'Error: {abs(val - Expected)}')
    data[f'M={fragmentation}'] = Errors
df = pd.DataFrame(data, index=[f'N={int(n * (i + 1 + size // 2) / size)}' for i in range(size)])
df

N = 6, M = 1000
Value: 0.2502017052487212
Expected: 0.25020169514301416
Error: 1.0105707048957413e-08


,M=600,M=800,M=1000,M=1200,M=1400
N=3,Error: 1.4394566566489075e-07,Error: 9.349358220944737e-08,Error: 6.689775844392898e-08,Error: 5.089044186057379e-08,Error: 4.038439244125058e-08
N=4,Error: 6.647569444906409e-08,Error: 4.3176736963523865e-08,Error: 3.089454581051143e-08,Error: 2.350217009894351e-08,Error: 1.8650323396762758e-08
N=6,Error: 2.1744202938211998e-08,Error: 1.41232004979841e-08,Error: 1.0105707326513169e-08,Error: 7.687656289334655e-09,Error: 6.1006079077330355e-09
N=7,Error: 1.410886979469339e-08,Error: 9.16394438110757e-09,Error: 6.557168918952527e-09,Error: 4.988200019706568e-09,Error: 3.958431915584981e-09
N=8,Error: 9.671229983521101e-09,Error: 6.281629549853562e-09,Error: 4.494759175965868e-09,Error: 3.419275640403896e-09,Error: 2.7133979507709682e-09


### Вывод узлов и коэффициентов исходной КФ Гаусса

In [64]:
# n вводится в предыдущем блоке
legendre = get_Legendre(n)
nodes = secant_method(legendre)
coefficients = get_gauss_coefficients(nodes, n)
data = {}
for i in range(n):
    data[f'Node {i+1}'] = {'Nodes': str(nodes[i]), 'Coefficients': str(coefficients[i])}
df = pd.DataFrame(data)
df

,Node 1,Node 2,Node 3,Node 4,Node 5,Node 6
Nodes,-0.932469514203152,-0.6612093864662646,-0.2386191860831969,0.23861918608319693,0.6612093864662645,0.932469514203152
Coefficients,0.171324492379171,0.360761573048138,0.46791393457269126,0.46791393457269126,0.3607615730481388,0.171324492379171
